In [1]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)

#### Fetch Mutual fund urls

In [ ]:
url_list=[]
for i in range(0,78):
    r=requests.get('https://groww.in/slr/v1/search/derived/scheme?available_for_investment=true&doc_type=scheme&max_aum=&page={}&plan_type=Direct&q=&size=15&sort_by=0'.format(str(i)))
    data=r.json()
    df2=pd.DataFrame(data['content'])
    df2.head(1)
    df2['url']='https://groww.in/mutual-funds/'+df2.search_id
    url_list.extend(list(df2['url']))

In [ ]:
url_list[0:20]

In [ ]:
len(url_list)

In [2]:
#### Fetch Mutual Data Investment information

In [ ]:
from bs4 import BeautifulSoup
import re
import json


In [ ]:
mutual_fund_data={}
for i in range(0,len(url_list)):
    r=requests.get(url_list[i])
    html_soup_object = BeautifulSoup(r.content, 'html.parser')
    matches=re.findall(r'window\.__INITIAL_STATE__.*<\/script>',r.text)
    filtered_data=matches[0].replace('window.__INITIAL_STATE__ = ','').replace('</script>','')
    holding_dict=json.loads(filtered_data)
    holding_dict['fundV2']['productPage']
    mutual_fund_name=list(holding_dict['fundV2']['productPage']['mfProductPageDataMap'].keys())[0]
    for hol_list in holding_dict['fundV2']['productPage']['mfProductPageDataMap'][mutual_fund_name]['MAIN_DATA']['holdings']:
        print(hol_list['company_name'],hol_list['sector_name'],hol_list['corpus_per'])
        if hol_list['company_name'] not in list(mutual_fund_data.keys()):
            mutual_fund_data[hol_list['company_name']]=[]
            mutual_fund_data[hol_list['company_name']].append([mutual_fund_name,hol_list['corpus_per']])
        else:
            mutual_fund_data[hol_list['company_name']].append([mutual_fund_name,hol_list['corpus_per']])
            
mutual_fund_data        

#### Run simple average operation on mutual fund data

In [ ]:
import statistics

In [ ]:
stats_mutual_data=[]
for key in mutual_fund_data.keys():
    #print(mutual_fund_data[key])
    average_data=[]
    for i in range(0,len(mutual_fund_data[key])):
        #print(mutual_fund_data[key][i][1])
        average_data.append(mutual_fund_data[key][i][1])
    #print(statistics.mean(average_data))    
        
    stats_mutual_data.append([len(mutual_fund_data[key]),key,statistics.mean(average_data)])
#stats_mutual_data 
stats_df=pd.DataFrame(stats_mutual_data,columns=['Number_of_mutual_fund','Company Name','Average_Contibution'])
stats_df.sort_values(['Number_of_mutual_fund','Average_Contibution'],ascending=[False, True]).head(60)